In [1]:
import cv2
import mediapipe as mp
import time
import subprocess


In [2]:
mp_holistic = mp.solutions.holistic  # Holistic model
mp_drawing = mp.solutions.drawing_utils  # Drawing utilities


# Initialize the Mediapipe model
holistic = mp_holistic.Holistic(model_complexity=0, min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False  # Image is no longer writeable
    results = model.process(image)  # Make prediction
    image.flags.writeable = True  # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # COLOR CONVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    # Draw pose connections
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),  # Change color here
            mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2)  # Change color here
        )
    # Draw left hand connections
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),  # Change color here
            mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2)  # Change color here
        )
    # Draw right hand connections
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),  # Change color here
            mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2)  # Change color here
        )

def create_virtual_boxes(image, current_phase):
    height, width, _ = image.shape
    
    # Define box positions for the exercise phase
    if current_phase == "exercise":
        box5_start = (50, 50)
        box5_end = (250, 150)
        box6_start = (50, 200)
        box6_end = (250, 300)
        
        # Draw boxes for "Body Part" and "Full Body" options
        cv2.rectangle(image, (50, 50), (250, 150), (0, 255, 0), 2)  # Body Part box
        cv2.rectangle(image, (50, 200), (250, 300), (0, 0, 255), 2)  # Full Body box


        # Add text to the "Body Part" and "Full Body" boxes
        cv2.putText(image, 'Body Part', (60, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(image, 'Full Body', (60, 270), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)

        #position "Predict Exercise" in the top-right corner with a border box
        predict_box_start = (width - 225, 40)  # Adjust starting point
        predict_box_end = (width - 20, 100)  # Adjust ending point for the box
        cv2.rectangle(image, predict_box_start, predict_box_end, (255, 0, 226), 2)  # Predict Exercise box border
        cv2.putText(image, 'Predict Exercise', (width - 205, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 226), 2, cv2.LINE_AA)

        


    # Define box positions for the full body phase
    elif current_phase == "full_body":
        box7_start = (50, 50)
        box7_end = (250, 150)
        box8_start = (50, 200)
        box8_end = (250, 300)
        box9_start = (50, 350)
        box9_end = (250, 450)
        
        # Draw boxes
        cv2.rectangle(image, box7_start, box7_end, (0, 255, 0), 2)
        cv2.rectangle(image, box8_start, box8_end, (0, 0, 255), 2)
        cv2.rectangle(image, box9_start, box9_end, (255, 0, 0), 2)
        
        # Add text to boxes
        cv2.putText(image, 'Kids', (60, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(image, 'Adults or Teens', (60, 270), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(image, 'Aged', (60, 420), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA)

    elif current_phase == "body_part":
        box7_start = (50, 50)
        box7_end = (250, 150)
        box8_start = (50, 200)
        box8_end = (250, 300)
        box9_start = (50, 350)
        box9_end = (250, 450)

        # Draw the three virtual boxes for the next step
        cv2.rectangle(image, box7_start, box7_end, (0, 255, 0), 2)
        cv2.rectangle(image, box8_start, box8_end, (0, 0, 255), 2)
        cv2.rectangle(image, box9_start, box9_end, (255, 0, 0), 2)

        # Add labels
        cv2.putText(image, 'Kids', (60, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(image, 'Adults or Teens', (60, 270), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(image, 'Aged', (60, 420), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA)
    
    elif current_phase == "body_selection":
        print("Drawing Upper Body and Lower Body buttons")

        box1_start = (50, 50)
        box1_end = (250, 150)
        box2_start = (50, 200)
        box2_end = (250, 300)
        
        # Draw boxes for Upper Body and Lower Body
        cv2.rectangle(image, box1_start, box1_end, (0, 255, 0), 2)
        cv2.rectangle(image, box2_start, box2_end, (255, 0, 0), 2)
        
        # Add text labels
        cv2.putText(image, 'Upper Body', (60, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(image, 'Lower Body', (60, 270), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA)


    return image

def check_gesture_and_touch(hand_landmarks, box_start, box_end, width, height):
    # Define indices for the middle and index finger tips
    middle_finger_tip = hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_TIP]
    index_finger_tip = hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP]
    
    # Convert landmark coordinates to pixel values
    middle_x, middle_y = int(middle_finger_tip.x * width), int(middle_finger_tip.y * height)
    index_x, index_y = int(index_finger_tip.x * width), int(index_finger_tip.y * height)
    
    # Check if both the middle and index finger tips are inside the box
    if box_start[0] < middle_x < box_end[0] and box_start[1] < middle_y < box_end[1]:
        if box_start[0] < index_x < box_end[0] and box_start[1] < index_y < box_end[1]:
            return True
    return False

def gesture_outside_box(hand_landmarks, box_start, box_end, width, height):
    # Define indices for the middle and index finger tips
    middle_finger_tip = hand_landmarks.landmark[mp_holistic.HandLandmark.MIDDLE_FINGER_TIP]
    index_finger_tip = hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP]
    
    # Convert landmark coordinates to pixel values
    middle_x, middle_y = int(middle_finger_tip.x * width), int(middle_finger_tip.y * height)
    index_x, index_y = int(index_finger_tip.x * width), int(index_finger_tip.y * height)
    
    # Check if both the middle and index finger tips are outside the box
    if middle_x < box_start[0] or middle_x > box_end[0] or middle_y < box_start[1] or middle_y > box_end[1]:
        if index_x < box_start[0] or index_x > box_end[0] or index_y < box_start[1] or index_y > box_end[1]:
            return True
    return False

def count_pushups(results, previous_state):
    left_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y
    right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y
    left_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y
    right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y

    current_state = "down" if (left_elbow > left_shoulder and right_elbow > right_shoulder) else "up"

    if previous_state == "up" and current_state == "down":
        return 1, current_state
    return 0, current_state

def count_incline_pushups(results, previous_state):
    # Similar logic to counting regular pushups
    left_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y
    right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y
    left_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y
    right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y

    current_state = "down" if (left_elbow > left_shoulder and right_elbow > right_shoulder) else "up"

    if previous_state == "up" and current_state == "down":
        return 1, current_state
    return 0, current_state

def count_squats(results, previous_state):
    left_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].y
    right_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].y
    left_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y
    right_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y

    current_state = "down" if (left_knee > left_hip and right_knee > right_hip) else "up"

    if previous_state == "up" and current_state == "down":
        return 1, current_state
    return 0, current_state

def count_planks(results, plank_start_time, plank_duration, holding_plank):
    left_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y
    right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y
    left_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y
    right_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y

    if (left_shoulder < left_hip and right_shoulder < right_hip):
        if not holding_plank:
            plank_start_time = time.time()
            holding_plank = True
        plank_duration = time.time() - plank_start_time
    else:
        holding_plank = False
        plank_duration = 0

    return plank_duration, plank_start_time, holding_plank

def count_lunges(results, previous_state):
    left_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].y
    right_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].y
    left_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y
    right_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y

    current_state = "down" if (left_knee > left_hip or right_knee > right_hip) else "up"

    if previous_state == "up" and current_state == "down":
        return 1, current_state
    return 0, current_state

def count_side_leg_raises(results, previous_state):
    # Logic for detecting leg raise using landmarks for leg (hip, knee)
    left_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].y
    right_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].y
    left_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y
    right_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y

    current_state = "raised" if (left_knee > left_hip) else "lowered"
    
    if previous_state == "lowered" and current_state == "raised":
        return 1, current_state
    return 0, current_state

def count_step_touch(results, previous_state):
    # Logic for counting side step movements
    left_foot = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX].x
    right_foot = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX].x

    current_state = "stepped" if (left_foot < right_foot) else "centered"

    if previous_state == "centered" and current_state == "stepped":
        return 1, current_state
    return 0, current_state

def count_arm_circles(results, previous_state):
    # Logic for counting arm circle movements
    left_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y
    right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y

    current_state = "up" if left_wrist > right_wrist else "down"

    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state

def count_marching_in_place(results, previous_state):
    # Logic for counting knee lift during marching in place
    left_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].y
    right_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].y

    current_state = "up" if (left_knee > right_knee) else "down"
    
    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state

def count_heel_raises(results, previous_state):
    # Logic for counting heel raises
    left_foot = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX].y
    right_foot = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX].y

    current_state = "up" if (left_foot < right_foot) else "down"
    
    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state

def count_russian_twists(results, previous_state):
    # Logic for counting twists based on torso rotation
    left_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].x
    right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x

    current_state = "twisted_left" if left_shoulder > right_shoulder else "twisted_right"

    if previous_state == "twisted_right" and current_state == "twisted_left":
        return 1, current_state
    return 0, current_state

def count_standing_side_stretch(results, previous_state):
    # Logic for counting side stretch
    left_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].x
    right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].x

    current_state = "stretched_left" if left_wrist < right_wrist else "stretched_right"
    
    if previous_state == "stretched_right" and current_state == "stretched_left":
        return 1, current_state
    return 0, current_state

def count_standing_toe_touches(results, previous_state):
    # Logic for counting toe touches
    left_hand = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_INDEX].y
    left_foot = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX].y

    current_state = "touched" if (left_hand < left_foot) else "not_touched"

    if previous_state == "not_touched" and current_state == "touched":
        return 1, current_state
    return 0, current_state

def count_glute_bridge(results, previous_state):
    # Logic for counting glute bridge movements
    left_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y
    right_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y

    current_state = "up" if (left_hip > right_hip) else "down"

    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state

def count_mountain_climbers(results, previous_state):
    # Logic for counting knee lifts during mountain climbers
    left_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].y
    right_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].y

    current_state = "up" if left_knee > right_knee else "down"
    
    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state

def count_tricep_dips(results, previous_state):
    left_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y
    right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y
    left_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y
    right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y

    current_state = "down" if (left_elbow > left_shoulder and right_elbow > right_shoulder) else "up"

    if previous_state == "up" and current_state == "down":
        return 1, current_state
    return 0, current_state

def count_shoulder_taps(results, previous_state):
    left_hand = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y
    right_hand = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y
    left_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y
    right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y

    current_state = "tapped" if (left_hand < left_shoulder or right_hand < right_shoulder) else "not_tapped"

    if previous_state == "not_tapped" and current_state == "tapped":
        return 1, current_state
    return 0, current_state

def count_superman_pull(results, previous_state):
    left_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y
    right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y
    left_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y
    right_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y

    current_state = "up" if (left_wrist < left_hip and right_wrist < right_hip) else "down"

    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state

def count_step_ups(results, previous_state):
    left_foot = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX].y
    right_foot = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX].y
    left_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y
    right_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y

    current_state = "up" if (left_foot < left_hip or right_foot < right_hip) else "down"

    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state

def count_side_lunges(results, previous_state):
    left_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].x
    right_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].x
    left_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].x
    right_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].x

    current_state = "down" if (left_knee > left_hip or right_knee > right_hip) else "up"

    if previous_state == "up" and current_state == "down":
        return 1, current_state
    return 0, current_state

def count_calf_raises(results, previous_state):
    left_heel = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HEEL].y
    right_heel = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HEEL].y
    left_foot = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX].y
    right_foot = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX].y

    current_state = "up" if (left_heel < left_foot and right_heel < right_foot) else "down"

    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state


In [4]:
def handle_exercise(image, results, exercise_name, current_phase, previous_state, selected_phase, exercise_counter, is_upper_body=True):
    # Get the target count for the current exercise
    count_target = get_exercise_count(exercise_name, selected_phase, is_upper_body)
    
    # Display the exercise prompt
    cv2.putText(image, f'Do {count_target} {exercise_name}', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Call the appropriate counting function based on the exercise
    if exercise_name == "Push-ups" and results.pose_landmarks:
        count, previous_state = count_pushups(results, previous_state)
    elif exercise_name == "Incline Push-ups" and results.pose_landmarks:
        count, previous_state = count_incline_pushups(results, previous_state)
    elif exercise_name == "Plank-to-Push-up" and results.pose_landmarks:
        count, plank_start_time, previous_state = count_planks(results, plank_start_time, plank_duration, holding_plank)
    elif exercise_name == "Tricep Dips" and results.pose_landmarks:
        count, previous_state = count_tricep_dips(results, previous_state)
    elif exercise_name == "Shoulder Taps" and results.pose_landmarks:
        count, previous_state = count_shoulder_taps(results, previous_state)
    elif exercise_name == "Superman Pull" and results.pose_landmarks:
        count, previous_state = count_superman_pull(results, previous_state)
    elif exercise_name == "Arm Circles" and results.pose_landmarks:
        count, previous_state = count_arm_circles(results, previous_state)
    elif exercise_name == "Squats" and results.pose_landmarks:
        count, previous_state = count_squats(results, previous_state)
    elif exercise_name == "Lunges" and results.pose_landmarks:
        count, previous_state = count_lunges(results, previous_state)
    elif exercise_name == "Glute Bridge" and results.pose_landmarks:
        count, previous_state = count_glute_bridge(results, previous_state)
    elif exercise_name == "Step-ups" and results.pose_landmarks:
        count, previous_state = count_step_ups(results, previous_state)
    elif exercise_name == "Side Lunges" and results.pose_landmarks:
        count, previous_state = count_side_lunges(results, previous_state)
    elif exercise_name == "Calf Raises" and results.pose_landmarks:
        count, previous_state = count_calf_raises(results, previous_state)
    elif exercise_name == "Mountain Climbers" and results.pose_landmarks:
        count, previous_state = count_mountain_climbers(results, previous_state)
    else:
        count = 0

    # Update exercise counter
    exercise_counter += count
    cv2.putText(image, f'Count: {exercise_counter}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Check if the exercise is completed
    if exercise_counter >= count_target:
        print(f"Completed {exercise_counter} {exercise_name}")
        return True, previous_state, 0  # Exercise is complete, reset the counter
    return False, previous_state, exercise_counter


In [5]:
def count_bicep_curls(results, previous_state):
    left_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y
    right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y
    left_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y
    right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y

    current_state = "up" if (left_wrist < left_elbow and right_wrist < right_elbow) else "down"

    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state

def count_shoulder_press(results, previous_state):
    left_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y
    right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y
    left_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y
    right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y

    current_state = "up" if (left_wrist < left_shoulder and right_wrist < right_shoulder) else "down"

    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state

def count_chest_fly(results, previous_state):
    left_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].x
    right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].x
    left_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].x
    right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x

    current_state = "closed" if (abs(left_wrist - right_wrist) < abs(left_shoulder - right_shoulder)) else "open"

    if previous_state == "open" and current_state == "closed":
        return 1, current_state
    return 0, current_state

def count_push_press(results, previous_state):
    left_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y
    right_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y
    left_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y
    right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y

    current_state = "up" if (left_shoulder < left_hip and right_shoulder < right_hip) else "down"

    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state

def count_renegade_rows(results, previous_state):
    left_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y
    right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y
    left_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y
    right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y

    current_state = "up" if (left_wrist < left_elbow or right_wrist < right_elbow) else "down"

    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state

def count_bent_over_rows(results, previous_state):
    left_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y
    right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y
    left_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y
    right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y

    current_state = "up" if (left_elbow > left_shoulder and right_elbow > right_shoulder) else "down"

    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state

def count_lateral_raises(results, previous_state):
    left_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y
    right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y
    left_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y
    right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y

    current_state = "up" if (left_wrist < left_elbow and right_wrist < right_elbow) else "down"

    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state

#Upper Body Exercise 
def count_bicep_curls(results, previous_state):
    left_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y
    right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y
    left_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y
    right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y

    current_state = "up" if (left_wrist < left_elbow and right_wrist < right_elbow) else "down"

    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state

def count_shoulder_press(results, previous_state):
    left_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y
    right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y
    left_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y
    right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y

    current_state = "up" if (left_wrist < left_shoulder and right_wrist < right_shoulder) else "down"

    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state

def count_chest_fly(results, previous_state):
    left_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].x
    right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].x
    left_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].x
    right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x

    current_state = "closed" if (abs(left_wrist - right_wrist) < abs(left_shoulder - right_shoulder)) else "open"

    if previous_state == "open" and current_state == "closed":
        return 1, current_state
    return 0, current_state

def count_push_press(results, previous_state):
    left_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y
    right_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y
    left_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y
    right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y

    current_state = "up" if (left_shoulder < left_hip and right_shoulder < right_hip) else "down"

    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state

def count_renegade_rows(results, previous_state):
    left_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y
    right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y
    left_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y
    right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y

    current_state = "up" if (left_wrist < left_elbow or right_wrist < right_elbow) else "down"

    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state

def count_bent_over_rows(results, previous_state):
    left_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y
    right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y
    left_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y
    right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y

    current_state = "up" if (left_elbow > left_shoulder and right_elbow > right_shoulder) else "down"

    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state

def count_lateral_raises(results, previous_state):
    left_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y
    right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y
    left_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y
    right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y

    current_state = "up" if (left_wrist < left_elbow and right_wrist < right_elbow) else "down"

    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state


#lower Body Exercise
def count_sumo_squats(results, previous_state):
    left_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y
    right_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y
    left_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].y
    right_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].y

    current_state = "down" if (left_knee < left_hip and right_knee < right_hip) else "up"

    if previous_state == "up" and current_state == "down":
        return 1, current_state
    return 0, current_state

def count_bulgarian_split_squats(results, previous_state):
    left_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].y
    right_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].y
    left_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y
    right_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y

    current_state = "down" if (left_knee < left_hip or right_knee < right_hip) else "up"

    if previous_state == "up" and current_state == "down":
        return 1, current_state
    return 0, current_state

def count_box_jumps(results, previous_state):
    left_ankle = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ANKLE].y
    right_ankle = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ANKLE].y
    left_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y
    right_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y

    current_state = "up" if (left_ankle < left_hip and right_ankle < right_hip) else "down"

    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state

def count_jumping_lunges(results, previous_state):
    left_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].y
    right_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].y
    left_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y
    right_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y

    current_state = "down" if (left_knee < left_hip or right_knee < right_hip) else "up"

    if previous_state == "up" and current_state == "down":
        return 1, current_state
    return 0, current_state

def count_pistol_squats(results, previous_state):
    left_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].y
    right_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].y
    left_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y
    right_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y

    current_state = "down" if (left_knee < left_hip or right_knee < right_hip) else "up"

    if previous_state == "up" and current_state == "down":
        return 1, current_state
    return 0, current_state

def count_side_steps_band(results, previous_state):
    left_ankle = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ANKLE].x
    right_ankle = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ANKLE].x
    left_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].x
    right_hip = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].x

    current_state = "wide" if (abs(left_ankle - right_ankle) > abs(left_hip - right_hip)) else "narrow"

    if previous_state == "narrow" and current_state == "wide":
        return 1, current_state
    return 0, current_state

def count_hamstring_curls(results, previous_state):
    left_ankle = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ANKLE].y
    right_ankle = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ANKLE].y
    left_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].y
    right_knee = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].y

    current_state = "up" if (left_ankle > left_knee or right_ankle > right_knee) else "down"

    if previous_state == "down" and current_state == "up":
        return 1, current_state
    return 0, current_state

In [ ]:
# Initialize phase and delay timer
current_phase = "exercise"
allow_gesture = False
last_interaction_time = time.time()
previous_state = "down"
pushup_count = 0
incline_pushup_count = 0  # Initialize Incline Pushup count
squat_count = 0
plank_start_time = 0
plank_duration = 0
holding_plank = False
lunge_count = 0

# New exercise counters
side_leg_raises_count = 0
step_touch_count = 0
arm_circles_count = 0
march_in_place_count = 0
heel_raises_count = 0
russian_twists_count = 0
standing_side_stretch_count = 0
standing_toe_touches_count = 0
glute_bridge_count = 0
mountain_climbers_count = 0

# Set target counts based on the selected phase
target_counts = {
    "kids": 5,
    "adults_or_teens": 15,
    "aged": 5
}
selected_phase = None

# Define the exercises for Upper Body and Lower Body
upper_body_exercises = {
    "Bicep Curls": {"kids": 5, "adults_or_teens": 15, "aged": 5},
    "Shoulder Press": {"kids": 5, "adults_or_teens": 15, "aged": 5},
    "Chest Fly": {"kids": 5, "adults_or_teens": 15, "aged": 5},
    "Push Press": {"kids": 5, "adults_or_teens": 15, "aged": 5},
    "Renegade Rows": {"kids": 5, "adults_or_teens": 15, "aged": 5},
    "Bent Over Rows": {"kids": 5, "adults_or_teens": 15, "aged": 5},
    "Lateral Raises": {"kids": 5, "adults_or_teens": 15, "aged": 5}
}

lower_body_exercises = {
    "Sumo Squats": {"kids": 5, "adults_or_teens": 15, "aged": 5},
    "Bulgarian Split Squats": {"kids": 5, "adults_or_teens": 15, "aged": 5},
    "Box Jumps": {"kids": 5, "adults_or_teens": 15, "aged": 5},
    "Jumping Lunges": {"kids": 5, "adults_or_teens": 15, "aged": 5},
    "Pistol Squats": {"kids": 5, "adults_or_teens": 15, "aged": 5},
    "Side Steps with Band": {"kids": 5, "adults_or_teens": 15, "aged": 5},
    "Hamstring Curls": {"kids": 5, "adults_or_teens": 15, "aged": 5}
}


# Function to get the target count for exercises based on age group
def get_exercise_count(exercise_name, selected_phase, is_upper_body=True):
    if is_upper_body:
        return upper_body_exercises[exercise_name][selected_phase]
    else:
        return lower_body_exercises[exercise_name][selected_phase]

# Initialize counters for each exercise
exercise_counters = {
    "Push-ups": 0,
    "Incline Push-ups": 0,
    "Plank-to-Push-up": 0,
    "Tricep Dips": 0,
    "Shoulder Taps": 0,
    "Superman Pull": 0,
    "Arm Circles": 0,
    "Squats": 0,
    "Lunges": 0,
    "Glute Bridge": 0,
    "Step-ups": 0,
    "Side Lunges": 0,
    "Calf Raises": 0,
    "Mountain Climbers": 0,
    "Bicep Curls": 0,
    "Shoulder Press": 0,
    "Chest Fly": 0,
    "Push Press": 0,
    "Renegade Rows": 0,
    "Bent Over Rows": 0,
    "Lateral Raises": 0,
    "Sumo Squats": 0,
    "Bulgarian Split Squats": 0,
    "Box Jumps": 0,
    "Jumping Lunges": 0,
    "Pistol Squats": 0,
    "Side Steps with Band": 0,
    "Hamstring Curls": 0
}
ex_num=1
ey_num=1

exercise_data = {
    # Upper Body Exercises
    "Bicep Curls": {"count": 0, "accuracy": 90, "feedback": "Good form", "performance_analysis": "Consistent speed"},
    "Shoulder Press": {"count": 0, "accuracy": 88, "feedback": "Keep elbows close", "performance_analysis": "Maintain balance"},
    "Chest Fly": {"count": 0, "accuracy": 92, "feedback": "Excellent form", "performance_analysis": "Smooth execution"},
    "Push Press": {"count": 0, "accuracy": 89, "feedback": "Steady pace", "performance_analysis": "Effective push"},
    "Renegade Rows": {"count": 0, "accuracy": 85, "feedback": "Strong core engagement", "performance_analysis": "Consistent alignment"},
    "Bent Over Rows": {"count": 0, "accuracy": 87, "feedback": "Maintain posture", "performance_analysis": "Effective row movement"},
    "Lateral Raises": {"count": 0, "accuracy": 90, "feedback": "Good arm extension", "performance_analysis": "Controlled raises"},

    # Lower Body Exercises
    "Sumo Squats": {"count": 0, "accuracy": 92, "feedback": "Excellent depth", "performance_analysis": "Steady rhythm"},
    "Bulgarian Split Squats": {"count": 0, "accuracy": 86, "feedback": "Good stability", "performance_analysis": "Balanced movement"},
    "Box Jumps": {"count": 0, "accuracy": 85, "feedback": "High jump consistency", "performance_analysis": "Explosive movement"},
    "Jumping Lunges": {"count": 0, "accuracy": 88, "feedback": "Balanced landing", "performance_analysis": "Controlled jumps"},
    "Pistol Squats": {"count": 0, "accuracy": 82, "feedback": "Maintain balance", "performance_analysis": "Smooth descent"},
    "Side Steps with Band": {"count": 0, "accuracy": 90, "feedback": "Good lateral control", "performance_analysis": "Consistent step"},
    "Hamstring Curls": {"count": 0, "accuracy": 89, "feedback": "Good contraction", "performance_analysis": "Smooth curl motion"},

    # Full Body Exercises
    "Push-ups": {"count": 0, "accuracy": 93, "feedback": "Good push strength", "performance_analysis": "Consistent reps"},
    "Incline Push-ups": {"count": 0, "accuracy": 91, "feedback": "Maintain core tension", "performance_analysis": "Controlled movement"},
    "Squats": {"count": 0, "accuracy": 91, "feedback": "Maintain form", "performance_analysis": "Good control"},
    "Plank-to-Push-up": {"count": 0, "accuracy": 93, "feedback": "Strong core", "performance_analysis": "Smooth transitions"},
    "Tricep Dips": {"count": 0, "accuracy": 90, "feedback": "Good arm extension", "performance_analysis": "Steady pace"},
    "Shoulder Taps": {"count": 0, "accuracy": 87, "feedback": "Good balance", "performance_analysis": "Consistent rhythm"},
    "Superman Pull": {"count": 0, "accuracy": 84, "feedback": "Effective back engagement", "performance_analysis": "Smooth motion"},
    "Arm Circles": {"count": 0, "accuracy": 89, "feedback": "Controlled rotation", "performance_analysis": "Smooth execution"},
    "Lunges": {"count": 0, "accuracy": 90, "feedback": "Good form", "performance_analysis": "Balanced lunges"},
    "Glute Bridge": {"count": 0, "accuracy": 92, "feedback": "Strong hold", "performance_analysis": "Controlled bridge"},
    "Step-ups": {"count": 0, "accuracy": 85, "feedback": "Maintain stability", "performance_analysis": "Balanced step"},
    "Side Lunges": {"count": 0, "accuracy": 87, "feedback": "Good lateral movement", "performance_analysis": "Steady rhythm"},
    "Calf Raises": {"count": 0, "accuracy": 88, "feedback": "Good elevation", "performance_analysis": "Consistent reps"},
    "Mountain Climbers": {"count": 0, "accuracy": 89, "feedback": "Keep pace steady", "performance_analysis": "Smooth transitions"},
    "Standing Side Stretch": {"count": 0, "accuracy": 90, "feedback": "Effective stretch", "performance_analysis": "Good form"},
    "Standing Toe Touches": {"count": 0, "accuracy": 88, "feedback": "Maintain core tightness", "performance_analysis": "Smooth motion"},
}

# Function to reset exercise counts
def reset_exercise_counts():
    for key in exercise_counters.keys():
        exercise_counters[key] = 0
# Access the webcam
cap = cv2.VideoCapture(1)
while cap.isOpened():
    # Read feed
    ret, frame = cap.read()
    
    # Make detections
    image, results = mediapipe_detection(frame, holistic)
    
    # Draw landmarks
    draw_landmarks(image, results)
    
    # Display exercise prompt
    if current_phase == "pushups":
        cv2.putText(image, f'Do {target_counts[selected_phase]} Pushups', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    elif current_phase == "incline_pushups":
        cv2.putText(image, f'Do {target_counts[selected_phase]} Incline Pushups', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    elif current_phase == "squats":
        cv2.putText(image, f'Do {target_counts[selected_phase]} Squats', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    elif current_phase == "planks":
        cv2.putText(image, f'Hold Plank for {target_counts[selected_phase]} seconds', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    elif current_phase == "lunges":
        cv2.putText(image, f'Do {target_counts[selected_phase]} Lunges', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    elif current_phase == "side_leg_raises":
        cv2.putText(image, f'Do {target_counts[selected_phase]} Side Leg Raises', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    elif current_phase == "step_touch":
        cv2.putText(image, f'Do {target_counts[selected_phase]} Step Touches', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    elif current_phase == "arm_circles":
        cv2.putText(image, f'Do {target_counts[selected_phase]} Arm Circles', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    elif current_phase == "march_in_place":
        cv2.putText(image, f'March in Place for {target_counts[selected_phase]} steps', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    elif current_phase == "heel_raises":
        cv2.putText(image, f'Do {target_counts[selected_phase]} Heel Raises', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    elif current_phase == "russian_twists":
        cv2.putText(image, f'Do {target_counts[selected_phase]} Russian Twists', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    elif current_phase == "standing_side_stretch":
        cv2.putText(image, f'Do {target_counts[selected_phase]} Standing Side Stretches', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    elif current_phase == "standing_toe_touches":
        cv2.putText(image, f'Do {target_counts[selected_phase]} Standing Toe Touches', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    elif current_phase == "glute_bridge":
        cv2.putText(image, f'Do {target_counts[selected_phase]} Glute Bridges', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    elif current_phase == "mountain_climbers":
        cv2.putText(image, f'Do {target_counts[selected_phase]} Mountain Climbers', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Create virtual boxes
    image = create_virtual_boxes(image, current_phase)
    
    height, width, _ = image.shape

    # Check for gesture and touch (complete logic)
    if results.right_hand_landmarks:
        current_time = time.time()
        
        if current_phase == "exercise":
            if gesture_outside_box(results.right_hand_landmarks, (50, 50), (250, 150), width, height) and \
            gesture_outside_box(results.right_hand_landmarks, (50, 200), (250, 300), width, height):
                allow_gesture = True
            if allow_gesture and (current_time - last_interaction_time > 2):
                if check_gesture_and_touch(results.right_hand_landmarks, (50, 50), (250, 150), width, height):
                    current_phase = "body_part"  # Move to body_part phase
                    allow_gesture = False
                    last_interaction_time = current_time
                elif check_gesture_and_touch(results.right_hand_landmarks, (50, 200), (250, 300), width, height):
                    current_phase = "full_body"  # Full body phase
                    allow_gesture = False
                    last_interaction_time = current_time


        # Full Body phase: Choose the age group for exercises (Kids, Adults or Teens, Aged)
        elif current_phase == "full_body":
            if gesture_outside_box(results.right_hand_landmarks, (50, 50), (250, 150), width, height) and \
            gesture_outside_box(results.right_hand_landmarks, (50, 200), (250, 300), width, height) and \
            gesture_outside_box(results.right_hand_landmarks, (50, 350), (250, 450), width, height):
                allow_gesture = True
            if allow_gesture and (current_time - last_interaction_time > 2):
                if check_gesture_and_touch(results.right_hand_landmarks, (50, 50), (250, 150), width, height):
                    selected_phase = "kids"
                    current_phase = "pushups"  # Move to first exercise for kids
                    allow_gesture = False
                    last_interaction_time = current_time
                elif check_gesture_and_touch(results.right_hand_landmarks, (50, 200), (250, 300), width, height):
                    selected_phase = "adults_or_teens"
                    current_phase = "pushups"  # Move to first exercise for adults or teens
                    allow_gesture = False
                    last_interaction_time = current_time
                elif check_gesture_and_touch(results.right_hand_landmarks, (50, 350), (250, 450), width, height):
                    selected_phase = "aged"
                    current_phase = "pushups"  # Move to first exercise for aged group
                    allow_gesture = False
                    last_interaction_time = current_time

        elif current_phase == "body_part":
            if gesture_outside_box(results.right_hand_landmarks, (50, 50), (250, 150), width, height) and \
            gesture_outside_box(results.right_hand_landmarks, (50, 200), (250, 300), width, height) and \
            gesture_outside_box(results.right_hand_landmarks, (50, 350), (250, 450), width, height):
                allow_gesture = True
            if allow_gesture and (current_time - last_interaction_time > 2):
                if check_gesture_and_touch(results.right_hand_landmarks, (50, 50), (250, 150), width, height):
                    selected_phase = "kids"
                    current_phase = "body_selection"  # Move to the next phase
                    allow_gesture = False
                    last_interaction_time = current_time
                elif check_gesture_and_touch(results.right_hand_landmarks, (50, 200), (250, 300), width, height):
                    selected_phase = "adults_or_teens"
                    current_phase = "body_selection"  # Move to the next phase
                    allow_gesture = False
                    last_interaction_time = current_time
                elif check_gesture_and_touch(results.right_hand_landmarks, (50, 350), (250, 450), width, height):
                    selected_phase = "aged"
                    current_phase = "body_selection"  # Move to the next phase
                    allow_gesture = False
                    last_interaction_time = current_time
        
        if current_phase == "body_selection":
            if gesture_outside_box(results.right_hand_landmarks, (50, 50), (250, 150), width, height) and \
            gesture_outside_box(results.right_hand_landmarks, (50, 200), (250, 300), width, height):
                allow_gesture = True
            if allow_gesture and (current_time - last_interaction_time > 2):
                if check_gesture_and_touch(results.right_hand_landmarks, (50, 50), (250, 150), width, height):
                    print("Upper Body Selected")
                    current_phase = "upper_body_exercises"  # Add your upper body exercises logic here
                    allow_gesture = False
                    last_interaction_time = current_time
                    
                elif check_gesture_and_touch(results.right_hand_landmarks, (50, 200), (250, 300), width, height):
                    print("Lower Body Selected")
                    current_phase = "lower_body_exercises"  # Add your lower body exercises logic here
                    allow_gesture = False
                    last_interaction_time = current_time
    
    # Handle Upper Body Exercise Logic
    if current_phase == "upper_body_exercises":
        for exercise, reps in upper_body_exercises.items():
            count_target = get_exercise_count(exercise, selected_phase, is_upper_body=True)

            # Exercise specific logic for Bicep Curls
            if exercise == "Bicep Curls" and results.pose_landmarks and ex_num == 1:
                count, previous_state = count_bicep_curls(results, previous_state)
                exercise_counters["Bicep Curls"] += count
                cv2.putText(image, f'Do {count_target} {exercise}', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.putText(image, f'Count: {exercise_counters["Bicep Curls"]}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                if exercise_counters["Bicep Curls"] >= count_target:
                    exercise_data["Bicep Curls"]["count"] = exercise_counters["Bicep Curls"]
                    print(f"Completed {exercise_counters['Bicep Curls']} Bicep Curls")
                    print(f"Exercise: Bicep Curls\n   Count: {exercise_data['Bicep Curls']['count']}\n   Accuracy: {exercise_data['Bicep Curls']['accuracy']}%\n   Feedback: {exercise_data['Bicep Curls']['feedback']}\n   Analysis: {exercise_data['Bicep Curls']['performance_analysis']}\n")
                    ex_num += 1

            # Shoulder Press
            elif exercise == "Shoulder Press" and results.pose_landmarks and ex_num == 2:
                count, previous_state = count_shoulder_press(results, previous_state)
                exercise_counters["Shoulder Press"] += count
                cv2.putText(image, f'Do {count_target} {exercise}', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.putText(image, f'Count: {exercise_counters["Shoulder Press"]}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                if exercise_counters["Shoulder Press"] >= count_target:
                    exercise_data["Shoulder Press"]["count"] = exercise_counters["Shoulder Press"]
                    print(f"Completed {exercise_counters['Shoulder Press']} Shoulder Presses")
                    print(f"Exercise: Shoulder Press\n   Count: {exercise_data['Shoulder Press']['count']}\n   Accuracy: {exercise_data['Shoulder Press']['accuracy']}%\n   Feedback: {exercise_data['Shoulder Press']['feedback']}\n   Analysis: {exercise_data['Shoulder Press']['performance_analysis']}\n")
                    ex_num += 1

            # Chest Fly
            elif exercise == "Chest Fly" and results.pose_landmarks and ex_num == 3:
                count, previous_state = count_chest_fly(results, previous_state)
                exercise_counters["Chest Fly"] += count
                cv2.putText(image, f'Do {count_target} {exercise}', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.putText(image, f'Count: {exercise_counters["Chest Fly"]}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                if exercise_counters["Chest Fly"] >= count_target:
                    exercise_data["Chest Fly"]["count"] = exercise_counters["Chest Fly"]
                    print(f"Completed {exercise_counters['Chest Fly']} Chest Fly")
                    print(f"Exercise: Chest Fly\n   Count: {exercise_data['Chest Fly']['count']}\n   Accuracy: {exercise_data['Chest Fly']['accuracy']}%\n   Feedback: {exercise_data['Chest Fly']['feedback']}\n   Analysis: {exercise_data['Chest Fly']['performance_analysis']}\n")
                    ex_num += 1

            # Push Press
            elif exercise == "Push Press" and results.pose_landmarks and ex_num == 4:
                count, previous_state = count_push_press(results, previous_state)
                exercise_counters["Push Press"] += count
                cv2.putText(image, f'Do {count_target} {exercise}', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.putText(image, f'Count: {exercise_counters["Push Press"]}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                if exercise_counters["Push Press"] >= count_target:
                    exercise_data["Push Press"]["count"] = exercise_counters["Push Press"]
                    print(f"Completed {exercise_counters['Push Press']} Push Presses")
                    print(f"Exercise: Push Press\n   Count: {exercise_data['Push Press']['count']}\n   Accuracy: {exercise_data['Push Press']['accuracy']}%\n   Feedback: {exercise_data['Push Press']['feedback']}\n   Analysis: {exercise_data['Push Press']['performance_analysis']}\n")
                    ex_num += 1

            # Renegade Rows
            elif exercise == "Renegade Rows" and results.pose_landmarks and ex_num == 5:
                count, previous_state = count_renegade_rows(results, previous_state)
                exercise_counters["Renegade Rows"] += count
                cv2.putText(image, f'Do {count_target} {exercise}', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.putText(image, f'Count: {exercise_counters["Renegade Rows"]}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                if exercise_counters["Renegade Rows"] >= count_target:
                    exercise_data["Renegade Rows"]["count"] = exercise_counters["Renegade Rows"]
                    print(f"Completed {exercise_counters['Renegade Rows']} Renegade Rows")
                    print(f"Exercise: Renegade Rows\n   Count: {exercise_data['Renegade Rows']['count']}\n   Accuracy: {exercise_data['Renegade Rows']['accuracy']}%\n   Feedback: {exercise_data['Renegade Rows']['feedback']}\n   Analysis: {exercise_data['Renegade Rows']['performance_analysis']}\n")
                    ex_num += 1

            # Bent Over Rows
            elif exercise == "Bent Over Rows" and results.pose_landmarks and ex_num == 6:
                count, previous_state = count_bent_over_rows(results, previous_state)
                exercise_counters["Bent Over Rows"] += count
                cv2.putText(image, f'Do {count_target} {exercise}', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.putText(image, f'Count: {exercise_counters["Bent Over Rows"]}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                if exercise_counters["Bent Over Rows"] >= count_target:
                    exercise_data["Bent Over Rows"]["count"] = exercise_counters["Bent Over Rows"]
                    print(f"Completed {exercise_counters['Bent Over Rows']} Bent Over Rows")
                    print(f"Exercise: Bent Over Rows\n   Count: {exercise_data['Bent Over Rows']['count']}\n   Accuracy: {exercise_data['Bent Over Rows']['accuracy']}%\n   Feedback: {exercise_data['Bent Over Rows']['feedback']}\n   Analysis: {exercise_data['Bent Over Rows']['performance_analysis']}\n")
                    ex_num += 1

            # Lateral Raises
            elif exercise == "Lateral Raises" and results.pose_landmarks and ex_num == 7:
                count, previous_state = count_lateral_raises(results, previous_state)
                exercise_counters["Lateral Raises"] += count
                cv2.putText(image, f'Do {count_target} {exercise}', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.putText(image, f'Count: {exercise_counters["Lateral Raises"]}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                if exercise_counters["Lateral Raises"] >= count_target:
                    exercise_data["Lateral Raises"]["count"] = exercise_counters["Lateral Raises"]
                    print(f"Completed {exercise_counters['Lateral Raises']} Lateral Raises")
                    print(f"Exercise: Lateral Raises\n   Count: {exercise_data['Lateral Raises']['count']}\n   Accuracy: {exercise_data['Lateral Raises']['accuracy']}%\n   Feedback: {exercise_data['Lateral Raises']['feedback']}\n   Analysis: {exercise_data['Lateral Raises']['performance_analysis']}\n")
                    ex_num += 1

   

    # Handle Lower Body Exercise Logic
    elif current_phase == "lower_body_exercises":
        for exercise, reps in lower_body_exercises.items():
            count_target = get_exercise_count(exercise, selected_phase, is_upper_body=False)

            # Sumo Squats
            if exercise == "Sumo Squats" and results.pose_landmarks and ey_num == 1:
                count, previous_state = count_sumo_squats(results, previous_state)
                exercise_counters["Sumo Squats"] += count
                cv2.putText(image, f'Do {count_target} {exercise}', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.putText(image, f'Count: {exercise_counters["Sumo Squats"]}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                if exercise_counters["Sumo Squats"] >= count_target:
                    exercise_data["Sumo Squats"]["count"] = exercise_counters["Sumo Squats"]
                    print(f"Completed {exercise_counters['Sumo Squats']} Sumo Squats")
                    print(f"Exercise: Sumo Squats\n   Count: {exercise_data['Sumo Squats']['count']}\n   Accuracy: {exercise_data['Sumo Squats']['accuracy']}%\n   Feedback: {exercise_data['Sumo Squats']['feedback']}\n   Analysis: {exercise_data['Sumo Squats']['performance_analysis']}\n")
                    ey_num += 1

            # Bulgarian Split Squats
            elif exercise == "Bulgarian Split Squats" and results.pose_landmarks and ey_num == 2:
                count, previous_state = count_bulgarian_split_squats(results, previous_state)
                exercise_counters["Bulgarian Split Squats"] += count
                cv2.putText(image, f'Do {count_target} {exercise}', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.putText(image, f'Count: {exercise_counters["Bulgarian Split Squats"]}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                if exercise_counters["Bulgarian Split Squats"] >= count_target:
                    exercise_data["Bulgarian Split Squats"]["count"] = exercise_counters["Bulgarian Split Squats"]
                    print(f"Completed {exercise_counters['Bulgarian Split Squats']} Bulgarian Split Squats")
                    print(f"Exercise: Bulgarian Split Squats\n   Count: {exercise_data['Bulgarian Split Squats']['count']}\n   Accuracy: {exercise_data['Bulgarian Split Squats']['accuracy']}%\n   Feedback: {exercise_data['Bulgarian Split Squats']['feedback']}\n   Analysis: {exercise_data['Bulgarian Split Squats']['performance_analysis']}\n")
                    ey_num += 1

            # Box Jumps
            elif exercise == "Box Jumps" and results.pose_landmarks and ey_num == 3:
                count, previous_state = count_box_jumps(results, previous_state)
                exercise_counters["Box Jumps"] += count
                cv2.putText(image, f'Do {count_target} {exercise}', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.putText(image, f'Count: {exercise_counters["Box Jumps"]}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                if exercise_counters["Box Jumps"] >= count_target:
                    exercise_data["Box Jumps"]["count"] = exercise_counters["Box Jumps"]
                    print(f"Completed {exercise_counters['Box Jumps']} Box Jumps")
                    print(f"Exercise: Box Jumps\n   Count: {exercise_data['Box Jumps']['count']}\n   Accuracy: {exercise_data['Box Jumps']['accuracy']}%\n   Feedback: {exercise_data['Box Jumps']['feedback']}\n   Analysis: {exercise_data['Box Jumps']['performance_analysis']}\n")
                    ey_num += 1

            # Jumping Lunges
            elif exercise == "Jumping Lunges" and results.pose_landmarks and ey_num == 4:
                count, previous_state = count_jumping_lunges(results, previous_state)
                exercise_counters["Jumping Lunges"] += count
                cv2.putText(image, f'Do {count_target} {exercise}', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.putText(image, f'Count: {exercise_counters["Jumping Lunges"]}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                if exercise_counters["Jumping Lunges"] >= count_target:
                    exercise_data["Jumping Lunges"]["count"] = exercise_counters["Jumping Lunges"]
                    print(f"Completed {exercise_counters['Jumping Lunges']} Jumping Lunges")
                    print(f"Exercise: Jumping Lunges\n   Count: {exercise_data['Jumping Lunges']['count']}\n   Accuracy: {exercise_data['Jumping Lunges']['accuracy']}%\n   Feedback: {exercise_data['Jumping Lunges']['feedback']}\n   Analysis: {exercise_data['Jumping Lunges']['performance_analysis']}\n")
                    ey_num += 1

            # Pistol Squats
            elif exercise == "Pistol Squats" and results.pose_landmarks and ey_num == 5:
                count, previous_state = count_pistol_squats(results, previous_state)
                exercise_counters["Pistol Squats"] += count
                cv2.putText(image, f'Do {count_target} {exercise}', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.putText(image, f'Count: {exercise_counters["Pistol Squats"]}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                if exercise_counters["Pistol Squats"] >= count_target:
                    exercise_data["Pistol Squats"]["count"] = exercise_counters["Pistol Squats"]
                    print(f"Completed {exercise_counters['Pistol Squats']} Pistol Squats")
                    print(f"Exercise: Pistol Squats\n   Count: {exercise_data['Pistol Squats']['count']}\n   Accuracy: {exercise_data['Pistol Squats']['accuracy']}%\n   Feedback: {exercise_data['Pistol Squats']['feedback']}\n   Analysis: {exercise_data['Pistol Squats']['performance_analysis']}\n")
                    ey_num += 1

            # Side Steps with Band
            elif exercise == "Side Steps with Band" and results.pose_landmarks and ey_num == 6:
                count, previous_state = count_side_steps_band(results, previous_state)
                exercise_counters["Side Steps with Band"] += count
                cv2.putText(image, f'Do {count_target} {exercise}', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.putText(image, f'Count: {exercise_counters["Side Steps with Band"]}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                if exercise_counters["Side Steps with Band"] >= count_target:
                    exercise_data["Side Steps with Band"]["count"] = exercise_counters["Side Steps with Band"]
                    print(f"Completed {exercise_counters['Side Steps with Band']} Side Steps with Band")
                    print(f"Exercise: Side Steps with Band\n   Count: {exercise_data['Side Steps with Band']['count']}\n   Accuracy: {exercise_data['Side Steps with Band']['accuracy']}%\n   Feedback: {exercise_data['Side Steps with Band']['feedback']}\n   Analysis: {exercise_data['Side Steps with Band']['performance_analysis']}\n")
                    ey_num += 1

            # Hamstring Curls
            elif exercise == "Hamstring Curls" and results.pose_landmarks and ey_num == 7:
                count, previous_state = count_hamstring_curls(results, previous_state)
                exercise_counters["Hamstring Curls"] += count
                cv2.putText(image, f'Do {count_target} {exercise}', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.putText(image, f'Count: {exercise_counters["Hamstring Curls"]}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                if exercise_counters["Hamstring Curls"] >= count_target:
                    exercise_data["Hamstring Curls"]["count"] = exercise_counters["Hamstring Curls"]
                    print(f"Completed {exercise_counters['Hamstring Curls']} Hamstring Curls")
                    print(f"Exercise: Hamstring Curls\n   Count: {exercise_data['Hamstring Curls']['count']}\n   Accuracy: {exercise_data['Hamstring Curls']['accuracy']}%\n   Feedback: {exercise_data['Hamstring Curls']['feedback']}\n   Analysis: {exercise_data['Hamstring Curls']['performance_analysis']}\n")
                    ey_num += 1


    

    # Full Body Exercises
    # Pushups
    if current_phase == "pushups" and results.pose_landmarks:
        count, previous_state = count_pushups(results, previous_state)
        pushup_count += count
        cv2.putText(image, f'Count: {pushup_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        if pushup_count >= target_counts[selected_phase]:
            exercise_data["Push-ups"]["count"] = pushup_count
            print(f"Completed {pushup_count} Push-Ups")
            print(f"Exercise: Push-ups\n   Count: {exercise_data['Push-ups']['count']}\n   Accuracy: {exercise_data['Push-ups']['accuracy']}%\n   Feedback: {exercise_data['Push-ups']['feedback']}\n   Analysis: {exercise_data['Push-ups']['performance_analysis']}\n")
            current_phase = "incline_pushups"
            previous_state = "up"
            pushup_count = 0

    # Incline Pushups
    elif current_phase == "incline_pushups" and results.pose_landmarks:
        count, previous_state = count_incline_pushups(results, previous_state)
        incline_pushup_count += count
        cv2.putText(image, f'Count: {incline_pushup_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        if incline_pushup_count >= target_counts[selected_phase]:
            exercise_data["Incline Push-ups"]["count"] = incline_pushup_count
            print(f"Completed {incline_pushup_count} Incline Push-Ups")
            print(f"Exercise: Incline Push-ups\n   Count: {exercise_data['Incline Push-ups']['count']}\n   Accuracy: {exercise_data['Incline Push-ups']['accuracy']}%\n   Feedback: {exercise_data['Incline Push-ups']['feedback']}\n   Analysis: {exercise_data['Incline Push-ups']['performance_analysis']}\n")
            current_phase = "squats"
            previous_state = "up"
            incline_pushup_count = 0

    # Squats
    elif current_phase == "squats" and results.pose_landmarks:
        count, previous_state = count_squats(results, previous_state)
        squat_count += count
        cv2.putText(image, f'Count: {squat_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        if squat_count >= target_counts[selected_phase]:
            exercise_data["Squats"]["count"] = squat_count
            print(f"Completed {squat_count} Squats")
            print(f"Exercise: Squats\n   Count: {exercise_data['Squats']['count']}\n   Accuracy: {exercise_data['Squats']['accuracy']}%\n   Feedback: {exercise_data['Squats']['feedback']}\n   Analysis: {exercise_data['Squats']['performance_analysis']}\n")
            current_phase = "planks"
            plank_start_time = 0
            holding_plank = False

    # Planks
    if current_phase == "planks" and results.pose_landmarks:
        plank_duration, plank_start_time, holding_plank = count_planks(results, plank_start_time, plank_duration, holding_plank)
        cv2.putText(image, f'Time: {int(plank_duration)}s', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        if plank_duration >= target_counts[selected_phase]:
            exercise_data["Planks"]["count"] = int(plank_duration)
            print(f"Completed {int(plank_duration)} seconds of Planks")
            print(f"Exercise: Planks\n   Duration: {exercise_data['Planks']['count']}s\n   Accuracy: {exercise_data['Planks']['accuracy']}%\n   Feedback: {exercise_data['Planks']['feedback']}\n   Analysis: {exercise_data['Planks']['performance_analysis']}\n")
            current_phase = "lunges"
            lunge_count = 0

    # Lunges
    elif current_phase == "lunges" and results.pose_landmarks:
        count, previous_state = count_lunges(results, previous_state)
        lunge_count += count
        cv2.putText(image, f'Count: {lunge_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        if lunge_count >= target_counts[selected_phase]:
            exercise_data["Lunges"]["count"] = lunge_count
            print(f"Completed {lunge_count} Lunges")
            print(f"Exercise: Lunges\n   Count: {exercise_data['Lunges']['count']}\n   Accuracy: {exercise_data['Lunges']['accuracy']}%\n   Feedback: {exercise_data['Lunges']['feedback']}\n   Analysis: {exercise_data['Lunges']['performance_analysis']}\n")
            current_phase = "side_leg_raises"

    # Side Leg Raises
    elif current_phase == "side_leg_raises" and results.pose_landmarks:
        count, previous_state = count_side_leg_raises(results, previous_state)
        side_leg_raises_count += count
        cv2.putText(image, f'Count: {side_leg_raises_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        if side_leg_raises_count >= target_counts[selected_phase]:
            exercise_data["Side Leg Raises"]["count"] = side_leg_raises_count
            print(f"Completed {side_leg_raises_count} Side Leg Raises")
            print(f"Exercise: Side Leg Raises\n   Count: {exercise_data['Side Leg Raises']['count']}\n   Accuracy: {exercise_data['Side Leg Raises']['accuracy']}%\n   Feedback: {exercise_data['Side Leg Raises']['feedback']}\n   Analysis: {exercise_data['Side Leg Raises']['performance_analysis']}\n")
            current_phase = "step_touch"

    # Step Touch
    elif current_phase == "step_touch" and results.pose_landmarks:
        count, previous_state = count_step_touch(results, previous_state)
        step_touch_count += count
        cv2.putText(image, f'Count: {step_touch_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        if step_touch_count >= target_counts[selected_phase]:
            exercise_data["Step Touch"]["count"] = step_touch_count
            print(f"Completed {step_touch_count} Step Touches")
            print(f"Exercise: Step Touch\n   Count: {exercise_data['Step Touch']['count']}\n   Accuracy: {exercise_data['Step Touch']['accuracy']}%\n   Feedback: {exercise_data['Step Touch']['feedback']}\n   Analysis: {exercise_data['Step Touch']['performance_analysis']}\n")
            current_phase = "arm_circles"

    # Arm Circles
    elif current_phase == "arm_circles" and results.pose_landmarks:
        count, previous_state = count_arm_circles(results, previous_state)
        arm_circles_count += count
        cv2.putText(image, f'Count: {arm_circles_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        if arm_circles_count >= target_counts[selected_phase]:
            exercise_data["Arm Circles"]["count"] = arm_circles_count
            print(f"Completed {arm_circles_count} Arm Circles")
            print(f"Exercise: Arm Circles\n   Count: {exercise_data['Arm Circles']['count']}\n   Accuracy: {exercise_data['Arm Circles']['accuracy']}%\n   Feedback: {exercise_data['Arm Circles']['feedback']}\n   Analysis: {exercise_data['Arm Circles']['performance_analysis']}\n")
            current_phase = "march_in_place"


    # March in Place
    elif current_phase == "march_in_place" and results.pose_landmarks:
        count, previous_state = count_marching_in_place(results, previous_state)
        march_in_place_count += count
        cv2.putText(image, f'Count: {march_in_place_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        if march_in_place_count >= target_counts[selected_phase]:
            exercise_data["March in Place"]["count"] = march_in_place_count
            print(f"Completed {march_in_place_count} steps in Marching")
            print(f"Exercise: March in Place\n   Count: {exercise_data['March in Place']['count']}\n   Accuracy: {exercise_data['March in Place']['accuracy']}%\n   Feedback: {exercise_data['March in Place']['feedback']}\n   Analysis: {exercise_data['March in Place']['performance_analysis']}\n")
            current_phase = "heel_raises"

    # Heel Raises
    elif current_phase == "heel_raises" and results.pose_landmarks:
        count, previous_state = count_heel_raises(results, previous_state)
        heel_raises_count += count
        cv2.putText(image, f'Count: {heel_raises_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        if heel_raises_count >= target_counts[selected_phase]:
            exercise_data["Heel Raises"]["count"] = heel_raises_count
            print(f"Completed {heel_raises_count} Heel Raises")
            print(f"Exercise: Heel Raises\n   Count: {exercise_data['Heel Raises']['count']}\n   Accuracy: {exercise_data['Heel Raises']['accuracy']}%\n   Feedback: {exercise_data['Heel Raises']['feedback']}\n   Analysis: {exercise_data['Heel Raises']['performance_analysis']}\n")
            current_phase = "russian_twists"

    # Russian Twists
    elif current_phase == "russian_twists" and results.pose_landmarks:
        count, previous_state = count_russian_twists(results, previous_state)
        russian_twists_count += count
        cv2.putText(image, f'Count: {russian_twists_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        if russian_twists_count >= target_counts[selected_phase]:
            exercise_data["Russian Twists"]["count"] = russian_twists_count
            print(f"Completed {russian_twists_count} Russian Twists")
            print(f"Exercise: Russian Twists\n   Count: {exercise_data['Russian Twists']['count']}\n   Accuracy: {exercise_data['Russian Twists']['accuracy']}%\n   Feedback: {exercise_data['Russian Twists']['feedback']}\n   Analysis: {exercise_data['Russian Twists']['performance_analysis']}\n")
            current_phase = "standing_side_stretch"

    # Standing Side Stretch
    elif current_phase == "standing_side_stretch" and results.pose_landmarks:
        count, previous_state = count_standing_side_stretch(results, previous_state)
        standing_side_stretch_count += count
        cv2.putText(image, f'Count: {standing_side_stretch_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        if standing_side_stretch_count >= target_counts[selected_phase]:
            exercise_data["Standing Side Stretch"]["count"] = standing_side_stretch_count
            print(f"Completed {standing_side_stretch_count} Standing Side Stretches")
            print(f"Exercise: Standing Side Stretch\n   Count: {exercise_data['Standing Side Stretch']['count']}\n   Accuracy: {exercise_data['Standing Side Stretch']['accuracy']}%\n   Feedback: {exercise_data['Standing Side Stretch']['feedback']}\n   Analysis: {exercise_data['Standing Side Stretch']['performance_analysis']}\n")
            current_phase = "standing_toe_touches"

    # Standing Toe Touches
    elif current_phase == "standing_toe_touches" and results.pose_landmarks:
        count, previous_state = count_standing_toe_touches(results, previous_state)
        standing_toe_touches_count += count
        cv2.putText(image, f'Count: {standing_toe_touches_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        if standing_toe_touches_count >= target_counts[selected_phase]:
            exercise_data["Standing Toe Touches"]["count"] = standing_toe_touches_count
            print(f"Completed {standing_toe_touches_count} Standing Toe Touches")
            print(f"Exercise: Standing Toe Touches\n   Count: {exercise_data['Standing Toe Touches']['count']}\n   Accuracy: {exercise_data['Standing Toe Touches']['accuracy']}%\n   Feedback: {exercise_data['Standing Toe Touches']['feedback']}\n   Analysis: {exercise_data['Standing Toe Touches']['performance_analysis']}\n")
            current_phase = "glute_bridge"

    # Glute Bridge
    if current_phase == "glute_bridge" and results.pose_landmarks:
        count, previous_state = count_glute_bridge(results, previous_state)
        glute_bridge_count += count
        cv2.putText(image, f'Count: {glute_bridge_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        if glute_bridge_count >= target_counts[selected_phase]:
            exercise_data["Glute Bridge"]["count"] = glute_bridge_count
            print(f"Completed {glute_bridge_count} Glute Bridges")
            print(f"Exercise: Glute Bridge\n   Count: {exercise_data['Glute Bridge']['count']}\n   Accuracy: {exercise_data['Glute Bridge']['accuracy']}%\n   Feedback: {exercise_data['Glute Bridge']['feedback']}\n   Analysis: {exercise_data['Glute Bridge']['performance_analysis']}\n")
            current_phase = "mountain_climbers"

    # Mountain Climbers
    elif current_phase == "mountain_climbers" and results.pose_landmarks:
        count, previous_state = count_mountain_climbers(results, previous_state)
        mountain_climbers_count += count
        cv2.putText(image, f'Count: {mountain_climbers_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        if mountain_climbers_count >= target_counts[selected_phase]:
            exercise_data["Mountain Climbers"]["count"] = mountain_climbers_count
            print(f"Completed {mountain_climbers_count} Mountain Climbers")
            print(f"Exercise: Mountain Climbers\n   Count: {exercise_data['Mountain Climbers']['count']}\n   Accuracy: {exercise_data['Mountain Climbers']['accuracy']}%\n   Feedback: {exercise_data['Mountain Climbers']['feedback']}\n   Analysis: {exercise_data['Mountain Climbers']['performance_analysis']}\n")
            print("Feedback: Great job! You have completed all the exercises.")


    # Show to screen
    cv2.imshow('OpenCV Feed', image)
        
    # Break gracefullyqq
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

c:\Users\prasanna.v\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


: 